# Content-based Filtering

Aproach:
- Table with features erstellen, ratings in zweitem table

- sklearn-kNN mit minkowski-distance auf features anwenden, erhalte matrix mit allen Nachbarn eines jeden Films inklusive Abstand

- Wähle für jeden Nutzer und Film die x-nächsten Nachbarn aus, die der Nutzer bewertet hat

- Funktion schreiben, die auf basis des ratings der neighbours das rating eines Filmes vorhersagt.

In [1]:
import pandas as pd
import numpy as np
import string
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import PCA
kwargs = dict(random_state=42)

In [2]:
movies = pd.read_csv('../../data/preprocessed/movies_id_updated.csv')
average_ratings = pd.read_csv('../../data/preprocessed/average_ratings.csv')
omdb = pd.read_csv('../../data/preprocessed/omdb_cleaned.csv')
ratings = pd.read_csv('../../data/preprocessed/ratings_clean_std_0.csv')

In [3]:
movies.head()

,id,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtPictureURL
0,1,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...
1,2,Jumanji,tt0113497,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995.0,1068044-jumanji,http://content8.flixster.com/movie/56/79/73/56...
2,3,Grumpy Old Men,tt0107050,Dos viejos gruñones,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,1993.0,grumpy_old_men,http://content6.flixster.com/movie/25/60/25602...
3,4,Waiting to Exhale,tt0114885,Esperando un respiro,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,1995.0,waiting_to_exhale,http://content9.flixster.com/movie/10/94/17/10...
4,5,Father of the Bride Part II,tt0113041,Vuelve el padre de la novia (Ahora también abu...,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,1995.0,father_of_the_bride_part_ii,http://content8.flixster.com/movie/25/54/25542...


In [4]:
ratings.head()

,Unnamed: 0,user_id,imdbID,rating
0,0,1264,tt0047034,3.5
1,1,213,tt0304141,2.5
2,2,593,tt0369436,3.0
3,3,609,tt1077258,4.0
4,4,1590,tt0052182,4.0


In [5]:
omdb.head()

,Title,Year,Rated,Runtime,Writer,Plot,Language,imdbRating,imdbVotes,imdbID,...,Series,Released_season,Released_month,Released_day,PG_Rating,Available_languages,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated
0,Toy Story,1995.0,G,81.0,"John Lasseter (original story by), Pete Docter...",A cowboy doll is profoundly threatened and jea...,['English'],8.3,852896.0,tt0114709,...,0,4.0,11.0,22.0,0.0,1,0.0,3.0,0.0,0.0
1,Jumanji,1995.0,PG,104.0,"Jonathan Hensleigh (screenplay by), Greg Taylo...",When two kids find and play a magical board ga...,"['English', ' French']",7.0,294340.0,tt0113497,...,0,1.0,12.0,15.0,1.0,2,0.0,0.0,0.0,0.0
2,Grumpy Old Men,1993.0,PG-13,103.0,Mark Steven Johnson,A lifelong feud between two neighbors since ch...,['English'],7.0,41401.0,tt0107050,...,0,1.0,12.0,25.0,2.0,1,0.0,0.0,0.0,0.0
3,Waiting to Exhale,1995.0,R,124.0,"Terry McMillan (novel), Terry McMillan (screen...","Based on Terry McMillan's novel, this film fol...",['English'],5.9,9222.0,tt0114885,...,0,1.0,12.0,22.0,3.0,1,0.0,0.0,0.0,0.0
4,Father of the Bride Part II,1995.0,PG,106.0,"Albert Hackett (screenplay ""Father's Little Di...",George Banks must deal not only with the pregn...,['English'],6.0,33005.0,tt0113041,...,0,1.0,12.0,8.0,1.0,1,0.0,0.0,0.0,1.0


In [6]:
omdb.columns

Index(['Title', 'Year', 'Rated', 'Runtime', 'Writer', 'Plot', 'Language',
       'imdbRating', 'imdbVotes', 'imdbID', 'Rotten Tomatoes', 'Metacritic',
       'Series', 'Released_season', 'Released_month', 'Released_day',
       'PG_Rating', 'Available_languages', 'Oscars_won', 'Oscars_nominated',
       'Golden_globe_won', 'Golden_globe_nominated'],
      dtype='object')

In [7]:
# Select relevant columns
omdb = omdb[['imdbID', 'Year', 'Runtime', 'Language', 'imdbRating', 'imdbVotes', 'Rotten Tomatoes', 'Metacritic',
       'Series', 'Released_month', 'PG_Rating', 'Available_languages', 'Oscars_won', 'Oscars_nominated',
       'Golden_globe_won', 'Golden_globe_nominated']]

In [8]:
omdb.isna().sum()

imdbID                       0
Year                         0
Runtime                      9
Language                     0
imdbRating                   4
imdbVotes                    5
Rotten Tomatoes           1237
Metacritic                4019
Series                       0
Released_month              56
PG_Rating                 2314
Available_languages          0
Oscars_won                   0
Oscars_nominated             0
Golden_globe_won             0
Golden_globe_nominated       0
dtype: int64

In [9]:
# Add movies which could not be retrieved by omdb
imdb_ids = pd.DataFrame(movies['imdbID'].unique()).rename(columns={0:'imdbID'})
omdb = imdb_ids.merge(omdb, how='left', on='imdbID')

In [10]:
# missing entries in omdb dataset
omdb[omdb['Year'].isna()]['imdbID']

707     tt1233247
2111    tt0800175
2241    tt0134948
3399    tt0257001
3810    tt0266860
4822    tt0720339
5118    tt0313487
7581    tt0439663
8140    tt0383974
9411    tt0182989
Name: imdbID, dtype: object

In [11]:
omdb.isna().sum()

imdbID                       0
Year                        10
Runtime                     15
Language                    10
imdbRating                  10
imdbVotes                   11
Rotten Tomatoes           1246
Metacritic                4025
Series                      10
Released_month              62
PG_Rating                 2320
Available_languages         10
Oscars_won                  10
Oscars_nominated            10
Golden_globe_won            10
Golden_globe_nominated      10
dtype: int64

In [12]:
# Replace NaN with 0 and handle accordingly for awards
for i in range (9,13):
    omdb.iloc[:,i] = omdb.iloc[:,i].fillna(0)

In [13]:
omdb.loc[:,'Series'] = omdb.loc[:,'Series'].fillna(0)


In [14]:
# Comment Max: Vorschlag - Stattdessen Median/Modalwert benutzen benutzen
# Fill NaN Series
omdb.loc[:,'Series'] = omdb.loc[:,'Series'].fillna(0)
# Fill NaN Runtime
omdb.loc[:,'Runtime'] = omdb.loc[:,'Runtime'].fillna(omdb.loc[:,'Runtime'].median())
# Fill median for all others
for i in range (4,16):
    omdb.iloc[:,i] = omdb.iloc[:,i].fillna(omdb.iloc[:,i].median())

In [22]:
# possibility to encode language as one-hot encoding
#mlb = MultiLabelBinarizer()
#mlb.fit_transform(merged_data['Language'])

In [44]:
# convert imdbID from string to float
omdb['imdbID'] = omdb['imdbID'].str.replace(r'tt','')
omdb['imdbID'] = omdb['imdbID'].astype(float)
omdb['imdbID'].unique()

In [47]:
# delete language for the moment as not yet encoded
omdb = omdb.drop(columns={'Language'})

In [52]:
# fill NaN of the Year
omdb['Year'] = omdb['Year'].fillna(omdb['Year'].median())

In [53]:
# show first entry
omdb.loc[omdb['imdbID'] == 114709]

,imdbID,Year,Runtime,imdbRating,imdbVotes,Rotten Tomatoes,Metacritic,Series,Released_month,PG_Rating,Available_languages,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated
0,114709.0,1995.0,81.0,8.3,852896.0,10.0,9.5,0.0,11.0,0.0,1.0,0.0,3.0,0.0,0.0


In [105]:
omdb.sort_values('imdbID').iloc[62,:]

imdbID                    20530.0
Year                       1929.0
Runtime                      16.0
imdbRating                    7.7
imdbVotes                 43871.0
Rotten Tomatoes              10.0
Metacritic                    5.9
Series                        0.0
Released_month                6.0
PG_Rating                     0.0
Available_languages           1.0
Oscars_won                    0.0
Oscars_nominated              0.0
Golden_globe_won              0.0
Golden_globe_nominated        0.0
Name: 7536, dtype: float64

In [88]:
# set index as imdbID in ascending order
features = omdb.set_index('imdbID', drop = True)
features = features.sort_index()

In [91]:
features

,Year,Runtime,imdbRating,imdbVotes,Rotten Tomatoes,Metacritic,Series,Released_month,PG_Rating,Available_languages,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated
imdbID,,,,,,,,,,,,,,
439.0,1903.0,11.0,7.3,16372.0,10.0,5.9,0.0,12.0,0.0,1.0,0.0,0.0,0.0,0.0
4972.0,1915.0,195.0,6.4,21600.0,9.3,5.9,0.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0
6864.0,1916.0,197.0,7.8,13578.0,9.7,9.9,0.0,10.0,0.0,1.0,0.0,0.0,0.0,0.0
8133.0,1917.0,30.0,7.7,7070.0,6.8,5.9,0.0,6.0,0.0,2.0,0.0,0.0,0.0,0.0
8395.0,1918.0,112.0,7.1,309.0,6.8,5.9,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1229827.0,2009.0,76.0,1.5,17198.0,2.4,4.5,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0
1233247.0,1994.0,101.0,6.7,9221.5,6.8,5.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1233381.0,2008.0,109.0,7.4,18584.0,7.7,7.3,0.0,9.0,0.0,1.0,0.0,0.0,0.0,0.0


In [92]:
# Standardize the features
scaler = preprocessing.StandardScaler()
features = scaler.fit_transform(features)

In [94]:
features

array([[-4.58007841, -3.8830774 ,  0.70698335, ..., -0.27359947,
        -0.06819758, -0.16894619],
       [-3.93326025,  3.71802885, -0.17612886, ..., -0.27359947,
        -0.06819758, -0.16894619],
       [-3.87935874,  3.80064957,  1.19760124, ..., -0.27359947,
        -0.06819758, -0.16894619],
       ...,
       [ 1.07958043,  0.16533789,  0.80510693, ..., -0.27359947,
        -0.06819758, -0.16894619],
       [ 1.07958043, -0.53693823,  1.98258987, ..., -0.27359947,
        -0.06819758, -0.16894619],
       [ 1.07958043, -1.85886976,  1.19760124, ..., -0.27359947,
        -0.06819758, -0.16894619]])

In [95]:
# features of the first entry (imdbID 439)
features[0]

array([-4.58007841, -3.8830774 ,  0.70698335, -0.24525673,  1.4084073 ,
        0.04927881, -0.0807199 ,  1.55476718, -1.33063277, -0.52609534,
       -0.20232346, -0.27359947, -0.06819758, -0.16894619])

In [110]:
features.shape[0]

9423

In [121]:
# create NearesNeighbor model with the minkowski distance
knn = NearestNeighbors(metric='minkowski', p=2, algorithm='brute', n_jobs=-1)

In [122]:
# fit to all the features
knn.fit(features)

NearestNeighbors(algorithm='brute', n_jobs=-1)

In [143]:
# create two array for distance and index to store
distances = np.empty((features.shape[0],features.shape[0]))
indices = np.empty((features.shape[0],features.shape[0]))

# iterate over all imdbIDs
for i in range(0, features.shape[0]):
    # get all neighbors and the distances for each imdbID i
    distances[i], indices[i] = knn.kneighbors([features[i]], n_neighbors=features.shape[0])
    
    if i%1000==0:
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [144]:
distances

array([[1.19209290e-07, 2.27544556e+00, 2.54714198e+00, ...,
        2.94028912e+01, 2.99112788e+01, 5.14335564e+01],
       [1.19209290e-07, 1.95373133e+00, 2.36633564e+00, ...,
        2.44255376e+01, 2.92037037e+01, 4.98646200e+01],
       [1.46000966e-07, 2.18348797e+00, 3.01868504e+00, ...,
        2.46147796e+01, 2.94861288e+01, 4.98854905e+01],
       ...,
       [0.00000000e+00, 5.49887872e-01, 5.69620176e-01, ...,
        2.62051235e+01, 2.86393407e+01, 5.01893923e+01],
       [0.00000000e+00, 1.18754646e+00, 1.22337721e+00, ...,
        2.65405535e+01, 2.86427378e+01, 5.02496493e+01],
       [0.00000000e+00, 1.26171467e+00, 1.42522736e+00, ...,
        2.58165283e+01, 3.10679987e+01, 4.90565538e+01]])

In [147]:
indices

array([[0.000e+00, 6.200e+01, 2.200e+01, ..., 8.993e+03, 6.383e+03,
        7.579e+03],
       [1.000e+00, 2.370e+02, 2.900e+01, ..., 8.381e+03, 8.993e+03,
        7.579e+03],
       [2.000e+00, 3.940e+02, 1.015e+03, ..., 3.507e+03, 8.993e+03,
        7.579e+03],
       ...,
       [9.420e+03, 7.686e+03, 8.956e+03, ..., 6.383e+03, 8.993e+03,
        7.579e+03],
       [9.421e+03, 8.401e+03, 9.251e+03, ..., 6.383e+03, 8.993e+03,
        7.579e+03],
       [9.422e+03, 7.177e+03, 8.343e+03, ..., 2.975e+03, 8.993e+03,
        7.579e+03]])

# Bis hier hin aktuell

In [ ]:
#function that computes the k-nearest neighbors of each movie based on the features
for i in omdb['imdbID'].unique():
    try:
        distances , indices = knn.kneighbors(omdb.loc[omdb['imdbID'] == i], n_neighbors=10)
    except ValueError:
        print('Ohoh, ValueError')

In [37]:
user_ratings['imdbID'] = user_ratings['imdbID'].astype(float)

In [38]:
user_ratings

,index,imdbID,user_id,rating,Year,Runtime,imdbRating,imdbVotes,Rotten Tomatoes,Metacritic,Series,Released_month,PG_Rating,Available_languages,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated
0,0,47034.0,1264,3.5,1954.0,96.0,7.6,27485.0,9.3,7.8,0.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,304141.0,213,2.5,2004.0,142.0,7.9,524511.0,9.0,8.2,0.0,6.0,1.0,1.0,0.0,2.0,0.0,0.0
2,2,369436.0,593,3.0,2008.0,88.0,5.7,61335.0,2.4,4.1,0.0,11.0,2.0,1.0,0.0,0.0,0.0,0.0
3,3,1077258.0,609,4.0,2007.0,105.0,7.1,196149.0,7.4,5.9,0.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0
4,4,52182.0,1590,4.0,1958.0,100.0,7.4,6337.0,6.9,5.9,0.0,2.0,0.0,1.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787267,787267,83530.0,1032,3.0,1982.0,85.0,6.2,47327.0,4.2,4.8,0.0,12.0,1.0,1.0,0.0,0.0,0.0,0.0
787268,787268,107798.0,99,3.0,1993.0,141.0,6.6,73544.0,5.3,5.0,0.0,12.0,2.0,1.0,0.0,0.0,0.0,0.0
787269,787269,93857.0,333,3.0,1987.0,88.0,5.1,12882.0,0.7,2.8,0.0,7.0,2.0,2.0,0.0,0.0,0.0,0.0
787270,787270,144168.0,49,3.0,1999.0,94.0,6.1,17926.0,7.1,6.5,0.0,8.0,2.0,1.0,0.0,0.0,0.0,0.0


In [39]:
user_ratings.dtypes

index                       int64
imdbID                    float64
user_id                     int64
rating                    float64
Year                      float64
Runtime                   float64
imdbRating                float64
imdbVotes                 float64
Rotten Tomatoes           float64
Metacritic                float64
Series                    float64
Released_month            float64
PG_Rating                 float64
Available_languages       float64
Oscars_won                float64
Oscars_nominated          float64
Golden_globe_won          float64
Golden_globe_nominated    float64
dtype: object

In [50]:
# function that computes a rating based on the neighbors
def compute_rating(neighbors, distances):
    # TODO: Function schreiben, die statt dem mean() der Nachbarn die Ratings auf Basis des Abstandes gewichtet
    # Weiter entfernte Filme werden geringer gewichtet nähere Filme werden stärker gewichtet
    
    pred = np.mean(neighbors['rating'])
    
    return pred

MemoryError: Unable to allocate 426. MiB for an array with shape (71, 787272) and data type float64

In [30]:
# function that computes a rating based on the neighbors
def compute_rating(neighbors, distances):
    # TODO: Function schreiben, die statt dem mean() der Nachbarn die Ratings auf Basis des Abstandes gewichtet
    # Weiter entfernte Filme werden geringer gewichtet nähere Filme werden stärker gewichtet
    
    pred = np.mean(neighbors['rating'])
    
    return pred

In [37]:
## function that predicts the rating of a movie from its imdbID and its 20 nearest neighbors
# calculate similarity not based on user ratings? (only movie features)

def predict_movie_rating(imdbID, userID, user_data):
    # get 20 (or x) nearest neighbors that were rated by userID
    # TODO: Use function from kNN.ipynb to find optimal k for algorithm 
    k_neighbors = 10    
    
    # Select all ratings given by User #userID
    ratings = user_data.loc[user_data['user_id'] == userID]
    
    # Get real rating -> remove this in the end -> currently done for validation
    real_ratings = ratings.loc[(ratings['imdbID'] == imdbID)]
    
    #remove real rating
    ratings = ratings[ratings['imdbID'] != imdbID] 
     
    
    if (ratings.to_numpy().size>0):   
        
        # TODO:
        # Convert to Sparse Matrix Format
        #ratings_as_csr = csr_matrix(user_ratings)
        
        # Set algorithm and params 
        # TODO adapt neighbors based on number of given ratings by user_id ?
        knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=k_neighbors, n_jobs=-1)

        # Training
        print('---- Training ConBF-kNN-Algorithm ----')
        print('user_id: '+str(userID))
        print('imdbID: '+str(imdbID))
        knn.fit(ratings)

        #Prediction -> get x nearest neighbors of imdbID
        distances , indices = knn.kneighbors(user_data.loc[user_data['imdbID'] == imdbID], n_neighbors=k_neighbors)
        
       
        # Collect neighbors by indices
        for i in range(0,indices.shape[1]):
            neighbors = ratings.iloc[[indices[0,i]]]
            
        # compute rating of movie(imbdID) based on the rating of the 20 nearest neighbors
        pred = compute_rating(neighbors, distances)
        
        #Generate Output for Understandability
        print('Predicted Rating for '+str(imdbID)+': '+str(pred))
        print('Real Rating of '+str(imdbID)+' was: '+ str(real_ratings['rating'].values[0]))
        
        # return rating prediction and real rating
        return pred , real_ratings['rating'].values[0]
        
    else:
         return "User has not rated other movies. Check input"
    

In [35]:
# Larger Test:
# TODO Refactoring needed #UglyPythonCode

def test_predict_mr(no_test_samples):
    # Sampling #no_test_samples of random samples from dataset for testing
    test_set = user_ratings.sample(n=no_test_samples)
    
    predictions = pd.DataFrame(columns=['Prediction'])
    reals = pd.DataFrame(columns=['Real_Rating'])
    
    # Iterate over test-set and generate predicitons for it
    # TODO get rid of ugly for-loop
    for row in test_set.itertuples():
        imdbID = row.imdbID
        userID = row.user_id
        pred , real = predict_movie_rating(imdbID, userID, user_ratings)
        predictions.loc[row.index] = pred
        reals.loc[row.index] = real
    
    rmse = mean_squared_error(reals['Real_Rating'], predictions['Prediction'], squared=False)
    print('RMSE: '+str(rmse))

In [40]:
n = 80
test_predict_mr(n)

---- Training ConBF-kNN-Algorithm ----
user_id: 376.0
imdbID: 120780.0
Predicted Rating for 120780.0: 1.0
Real Rating of 120780.0 was: 4.0
---- Training ConBF-kNN-Algorithm ----
user_id: 1431.0
imdbID: 94721.0
Predicted Rating for 94721.0: 4.0
Real Rating of 94721.0 was: 4.5
---- Training ConBF-kNN-Algorithm ----
user_id: 337.0
imdbID: 190641.0
Predicted Rating for 190641.0: 2.5
Real Rating of 190641.0 was: 4.0
---- Training ConBF-kNN-Algorithm ----
user_id: 361.0
imdbID: 94336.0
Predicted Rating for 94336.0: 4.0
Real Rating of 94336.0 was: 4.5
---- Training ConBF-kNN-Algorithm ----
user_id: 340.0
imdbID: 95016.0
Predicted Rating for 95016.0: 3.5
Real Rating of 95016.0 was: 3.0
---- Training ConBF-kNN-Algorithm ----
user_id: 70.0
imdbID: 80684.0
Predicted Rating for 80684.0: 4.5
Real Rating of 80684.0 was: 4.0
---- Training ConBF-kNN-Algorithm ----
user_id: 488.0
imdbID: 117119.0
Predicted Rating for 117119.0: 4.5
Real Rating of 117119.0 was: 4.0
---- Training ConBF-kNN-Algorithm ----
